# **Supervised Machine Learning - Final Assignment**
[Mathijs de Jong (380891)](mailto:m.de.jong@tinbergen.nl)

---
---

## **Environment Setup**

#### **Initialize notebook settings**

In [1]:
################################################################################
# Initialize notebook settings
################################################################################

# Install and upgrade required Python packages
!pip -q install pip rpy2 --upgrade

# Load R support into Jupyter notebook
import rpy2
%reload_ext rpy2.ipython

#### **Initialize R environment settings**

In [2]:
%%R
################################################################################
# Initialize R environment settings
################################################################################

# Initialize working directory
BASE.DIR = '/Users/mathijs/Google Drive/Tinbergen - MPhil'
setwd(paste0(BASE.DIR, '/Supervised Machine Learning/Final assignment'))

# Specify printing settings and set seed
options(scipen=999)
set.seed(42)

#### **Importing R dependencies**

In [3]:
%%R
################################################################################
# Load dependencies
################################################################################

# Install public packages
install.packages(setdiff(c('censReg', 'caret', 'devtools', 'doParallel',
  'lmtest', 'randomForest', 'rpart', 'sampleSelection', 'sandwich'),
  installed.packages()), verbose=F, quiet=T,
  repos='https://mirror.lyrahosting.com/CRAN/')

# Install and load latest version of own (mlkit) package
devtools::install_github('Accelerytics/mlkit', upgrade='always', force=T,
  quiet=T)

## **Data Pre-Processing**

In [4]:
%%R
################################################################################
# Pre-process data
################################################################################

# Load and pre-process 
df.raw = readxl::read_excel("dataset13.xlsx")
df = df.raw[df.raw$mail == 1, ]

# Update and add new explanatory variables
for (i in nrow(df))
  if (df[i, 'mailid'] != 7) df[i, 'lastresp'] = df[i - 1, 'resp']

N.all = nrow(df)

In [5]:
%%R
# Specify models
y.amount = ifelse(df$amount > 0, log(df$amount), 0)
y.resp = df$resp
form = formula(amount ~ 1 + lastresp + avresp + avamount + urblvl + hhsize
  + lowinc + highinc)
select.form = formula(resp ~ 1 + lastresp + avresp + urblvl + hhsize
  + lowinc + highinc)
amount.form = formula(amount ~ 1 + lastresp + avamount + urblvl + hhsize
  + lowinc + highinc)

# Create train, develop and test sets
train.ids = 
dev.ids = 
test.ids = 

x = model.matrix(form, data=df)
x.select = model.matrix(select.form, data=df)
x.amount = model.matrix(amount.form, data=df)

## **Benchmark Models**

#### **Benchmark: Tobit I**

In [6]:
%%R
################################################################################
# Benchmark model - Tobit I
################################################################################

# Tobit I - MLE using censReg package
censReg.tobit1 = censReg::censReg(form, left=0, right=Inf, data=df)
print(summary(censReg.tobit1))

# Tobit I - MLE using own implementation
tobit1_log_likelihood = function(theta, y, x) {
  length.theta = length(theta)
  sigma = exp(theta[length.theta])
  x.beta = x %*% theta[1:(length.theta - 1)]
  return(sum(!y * pnorm(-x.beta / sigma, log.p=T) - (y > 0) * (0.5 * log(2 * pi)
    + log(sigma) + ((y - x.beta) ^ 2) / (2 * sigma ^ 2))))
}
theta0 = censReg.tobit1$estimate
ml.tobit1 = optim(
  par = theta0,
  fn = function(theta) return(-tobit1_log_likelihood(theta, y.amount, x))
)
print(ml.tobit1$par)


Call:
censReg::censReg(formula = form, left = 0, right = Inf, data = df)

Observations:
         Total  Left-censored     Uncensored Right-censored 
         46163          28486          17677              0 

Coefficients:
              Estimate Std. error t value              Pr(> t)    
(Intercept) -32.800926   3.582929  -9.155 < 0.0000000000000002 ***
lastresp     -4.388837   0.423768 -10.357 < 0.0000000000000002 ***
avresp        0.309710   0.006084  50.903 < 0.0000000000000002 ***
avamount      0.404889   0.007632  53.050 < 0.0000000000000002 ***
urblvl        0.137276   0.104208   1.317             0.187729    
hhsize       -0.013356   0.732363  -0.018             0.985450    
lowinc        0.095831   0.044138   2.171             0.029919 *  
highinc       0.118516   0.034898   3.396             0.000684 ***
logSigma      3.209859   0.005910 543.126 < 0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Newton-Raphson maximisation, 7 itera

#### **Benchmark: Tobit II**

In [7]:
%%R
################################################################################
# Benchmark model - Tobit II
################################################################################

# Tobit II - Heckman Two-Step Procedure using sampleSelection package
two.step.tobit2.res = sampleSelection::selection(
  y.resp ~ 0 + x.select,
  y.amount ~ 0 + x.amount,
  method='2step'
)
print(summary(two.step.tobit2.res))

# Tobit II - MLE using sampleSelection package
ml.tobit2.res = sampleSelection::selection(
  y.resp ~ 0 + x.select,
  y.amount ~ 0 + x.amount,
  method='ml'
)
print(summary(ml.tobit2.res))

# Tobit II - Heckman Two-Step Procedure using own implementation
select.probit = glm(y.resp ~ 0 + x.select, family=binomial(link="probit"))
summary(select.probit)
x.beta.select = x.select %*% select.probit$coefficients
inv.mills = dnorm(x.beta.select) / pnorm(x.beta.select)
ids = y.amount != 0
amount.res = lm(y.amount[ids] ~ 0 + x.amount[ids, ] + inv.mills[ids])
print(lmtest::coeftest(amount.res, vcov=sandwich::vcovHC(amount.res,
  type="HC0")))
eta = amount.res$residuals
sigma12 = amount.res$coefficients[length(amount.res$coefficients)]
sigma22 = sum(eta ^ 2 + sigma12 ^ 2 * inv.mills[ids] * (x.beta.select[ids]
  + inv.mills[ids])) / (sum(ids) - 1)
sqrt.sigma22 = sqrt(sigma22)
rho = sigma12 / sqrt.sigma22
res.df = data.frame('estimate' = c(sqrt.sigma22, rho))
rownames(res.df) = c('sigma', 'rho')
print(res.df)

# Tobit II - MLE using own implementation
theta0 = c(sqrt.sigma22, rho, select.probit$coefficients,
  amount.res$coefficients[1:ncol(x.amount)])
names(theta0) = c('sigma', 'rho', paste0('x.select.', colnames(x.select)),
  paste0('x.amount.', colnames(x.amount)))

tobit2_log_likelihood = function(theta, y, x.select, x.amount) {
  sigma = theta[1]
  rho = theta[2]
  d = (y == 0)
  k.select = ncol(x.select)
  x.beta.select = x.select %*% theta[3:(2 + k.select)]
  x.beta.amount = x.amount %*% theta[(3 + k.select):length(theta)]
  return(sum(!d * pnorm(-x.beta.select, log.p=T) + d * pnorm(x.beta.select
    + rho / sigma * (y - x.beta.amount) / sqrt(1 - rho ^ 2), log.p=T)
    - 0.5 * log(2 * pi) - log(sigma) - ((y - x.beta.amount) ^ 2
    / (2 * sigma ^ 2))))
}

ml.tobit2 = optim(
  par = theta0,
  fn = function(theta) return(-tobit2_log_likelihood(theta, y.amount, x.select,
  x.amount))
)
print(ml.tobit2$par)

--------------------------------------------
Tobit 2 model (sample selection model)
2-step Heckman / heckit estimation
46163 observations (28486 censored and 17677 observed)
17 free parameters (df = 46147)
Probit selection equation:
                      Estimate Std. Error t value             Pr(>|t|)    
x.select(Intercept) -1.3974933  0.1620550  -8.624 < 0.0000000000000002 ***
x.selectlastresp    -0.2033947  0.0189264 -10.747 < 0.0000000000000002 ***
x.selectavresp       0.0184140  0.0002605  70.684 < 0.0000000000000002 ***
x.selecturblvl      -0.0072820  0.0047415  -1.536              0.12459    
x.selecthhsize       0.0041912  0.0332783   0.126              0.89978    
x.selectlowinc       0.0057073  0.0020044   2.847              0.00441 ** 
x.selecthighinc      0.0050809  0.0015857   3.204              0.00135 ** 
Outcome equation:
                      Estimate Std. Error t value             Pr(>|t|)    
x.amount(Intercept)  1.7040338  0.1097294  15.529 < 0.0000000000000002 ***

## **Two-Part Models**

In [8]:
%%R
cl = parallel::makePSOCKcluster(5)
doParallel::registerDoParallel(cl)

In [9]:
%%R
classifier.ctrl = caret::trainControl(
  method = 'cv',
  number = 5,
  search = 'grid'
)

regression.ctrl = caret::trainControl(
  method = 'cv',
  number = 5,
  search = 'grid'
)

mixed.ctrl = caret::trainControl(
  method = 'cv',
  number = 5,
  search = 'grid'
)

In [16]:
%%R
classifiers = list(
  glm.logit = list(
    method = 'glm',
    grid = NULL,
    args = list(family = 'binomial(link="logit")')
  ),
  'glm.probit' = list(
    method = 'glm',
    grid = NULL,
    args = list(family = 'binomial(link="gaussian")')
  ), 
  'svm.linear' = list(method='lssvmLinear', grid=expand.grid(
    tau = c(0, 1, 2, 3)
  )),
  'svm.poly' = list(method='lssvmPoly', grid=expand.grid(
    degree = c(),
    scale = c(),
    tau = c()
  )),
  'svm.radial' = list(method='lssvmRadial', grid=expand.grid(
    sigma = c(),
    tau = c()
  )),
  'tree.adaboost' = list(method='adaboost', grid=expand.grid(
    nIter = c(),
    method = c()
  )),
  'tree.logitBoost' = list(method='LogitBoost', grid=expand.grid(
    nIter = c()
  )),
  'tree.gbm' = list(method='gbm', grid=expand.grid(
    n.trees = c(),
    iteraction.depth = c(),
    shrinkage = c(),
    n.minobsinnode = c()
  ))
)

# regression.models = list(
#   'glm.net' : list(method='glmnet', c(alpha, lambda)),
#   'krr.poly' : list(method='krlsPoly', c(lambda, degree)),
#   'krr.radial' : list(method='krlsRadial', c(lambda, sigma))
#   'cart.standard' : list(method='rpart', c(cp))
#   'xgboost.linear' : list(method='xgbLinear', c(nrounds, lambda, alpha, eta))
#   'xgboost.tree' : list(method='xgbTree', c(nrounds, max_depth, eta, gamma, colsample_bytree, min_child_weight, subsample))
# )

# mixed.models = list(
#   'xgboost.linear' : list(method='xgbLinear', c(nrounds, lambda, alpha, eta))
#   'xgboost.tree' : list(method='xgbTree', c(nrounds, max_depth, eta, gamma, colsample_bytree, min_child_weight, subsample))
# )

In [ ]:
%%R
params.length = 3
params.list = list(
  lambda = 2 ^ seq(5, -5, length.out = 10),
  kernel = c(kernlab::vanilladot, kernlab::polydot, kernlab::rbfdot),
  kernel.offset = 1,
  kernel.degree = c(0.5, 2, 3),
  kernel.sigma = c(0.5, 1, 2),
  kernel.scale = 1,
  hinge = c('absolute', 'quadratic', 'huber'),
  hinge.delta = c(0.5, 2, 3)
)

# Specify fold ids
N = nrow(df)
n.folds = 3
fold.id = ((1:N) %% n.folds + 1)[sample(N, N)]

# Define metric functions
miss.rate = function(y.hat, y) sum(y != ifelse(y.hat >= 0, 1L, -1L)) / length(y)

# Grid search 3-fold cross-validation
gscv.svm = mlkit::grid.search.cross.validation(select.form, df, SVMMaj::svmmaj,
  params.list, ind.metric=miss.rate, fold.id=fold.id, force=T, verbose=T,
  use.formula=F, scale='zscore')

From cffi callback <function _processevents at 0x7f8583fda040>:
Traceback (most recent call last):
  File "/Users/mathijs/tinbergen/lib/python3.8/site-packages/rpy2/rinterface_lib/callbacks.py", line 270, in _processevents
    try:
KeyboardInterrupt
From cffi callback <function _processevents at 0x7f8583fda040>:
Traceback (most recent call last):
  File "/Users/mathijs/tinbergen/lib/python3.8/site-packages/rpy2/rinterface_lib/callbacks.py", line 270, in _processevents
    try:
KeyboardInterrupt
From cffi callback <function _processevents at 0x7f8583fda040>:
Traceback (most recent call last):
  File "/Users/mathijs/tinbergen/lib/python3.8/site-packages/rpy2/rinterface_lib/callbacks.py", line 270, in _processevents
    try:
KeyboardInterrupt
From cffi callback <function _processevents at 0x7f8583fda040>:
Traceback (most recent call last):
  File "/Users/mathijs/tinbergen/lib/python3.8/site-packages/rpy2/rinterface_lib/callbacks.py", line 270, in _processevents
    try:
KeyboardInterrupt


In [18]:
%%R
train.classifiers = function(classifiers, df.train, form, df.test=df.train) {
  dep.var = all.vars(form)[1]
  y.train = ifelse(df.train[, dep.var] > 0, 1L, 0L)
  
  for (classifier in names(classifiers)) {
    
    # Train the classifier
    classifiers[[classifier]][['model']] = caret::train(
      x = model.matrix(form, df.train),
      y = y.train,
      method = classifiers[[classifier]][['method']],
#       ... = classifiers[[classifier]][['args']],
      metric = 'Accuracy'
#       trControl = classifier.ctrl,
#       tuneGrid = classifiers[[classifier]][['grid']]
    )

    # Extract the performance metrics on the test data and store them
#     classifiers[[classifier]][['metrics']] = caret::confusionMatrix(
#       data = predict(
#         classifiers[[classifier]][['model']],
#         newdata = model.matrix(form, df.test)
#       ),
#       reference = df.test[, dep.var],
#       mode = "prec_recall"
#     )
  }
  return(classifiers)
}

classifiers = train.classifiers(classifiers, df, select.form)

R[write to console]: Error in y[, "time"] : subscript out of bounds




Error in y[, "time"] : subscript out of bounds


#### **Binary Classifier: Probit Regression**

#### **Binary Classifier: Logit Regression**

#### **Binary Classifier: Support Vector Machines**

#### **Binary Classifier: Decision Tree**

#### **Binary Classifier: Random Forest**

#### **Binary Classifier: AdaBoost.M1**

In [ ]:
%%R
################################################################################
# Two-Part Models - Classifiers
################################################################################

# Logit Classifier
select.logit = glm(y.resp ~ 0 + x.select, family=binomial(link="logit"))

# SVM
# Specify hyperparameter values to consider
params.length = 3
params.list = list(
  lambda = 2 ^ seq(5, -5, length.out = 10),
  kernel = c(kernlab::vanilladot, kernlab::polydot, kernlab::rbfdot),
  kernel.offset = 1,
  kernel.degree = c(0.5, 2, 3),
  kernel.sigma = c(0.5, 1, 2),
  kernel.scale = 1,
  hinge = c('absolute', 'quadratic', 'huber'),
  hinge.delta = c(0.5, 2, 3)
)

# Specify fold ids
N = nrow(df.train); n.folds = 3; fold.id = ((1:N) %% n.folds + 1)[sample(N, N)]

################################################################################
# Hyperparameter tuning
################################################################################

# Define metric functions
miss.rate = function(y.hat, y) sum(y != ifelse(y.hat >= 0, 1L, -1L)) / length(y)

# Grid search 3-fold cross-validation
gscv.svm = grid.search.cross.validation(formula, df.train, SVMMaj::svmmaj,
                                        params.list, ind.metric=mis.rate, fold.id=fold.id, force=T, verbose=T,
                                        use.formula=F, scale='zscore')

res = SVMMaj::svmmaj(X.train, y.train, lambda = 14, scale='zscore',
                     hinge='quadratic')
pred = predict(res, X.test)
  
# Decision tree
select.tree = rpart::rpart(select.form, data=df,
  control=rpart::rpart.control(cp=0.00001))

# Random forest
randomForest::randomForest(select.form, data=df, mtry = 3, ntree = 500)

#### **Regression: Elastic Net Regression**

#### **Regression: Kernel Ridge Regression**

#### **Regression: CART**

#### **Regression: Random Forest**

#### **Regression: XGBoost**

## **Direct Models**

#### **Censored Regression: Decision Tree**

#### **Censored Regression: Random Forest**

#### **Censored Regression: XGBoost**

In [ ]:
################################################################################
# Two-Part Models - Mixed
################################################################################

# Decision tree
select.tree = rpart::rpart(y.amount ~ 0 + x, control=rpart::rpart.control(
  cp=0.00001))

# Random forest
randomForest::randomForest(y.amount ~ 0 + x, mtry = 3, ntree = 500)

In [64]:
%%R
parallel::stopCluster(cl)